<a href="https://colab.research.google.com/github/Khunapat/Exxonmobile-Hackhaton/blob/main/Cost_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seperate Inventory by month

In [ ]:
import pandas as pd
import os

# Load the Inventory data (replace the path with your file path in Colab)
file_path = '/content/Inventory.csv'  # Update this with the correct path
inventory_data = pd.read_csv(file_path, parse_dates=['BALANCE_AS_OF_DATE'])

# Create a new directory to store the separated files
output_directory = '/content/Inventory'  # Update this with your desired path
os.makedirs(output_directory, exist_ok=True)

# Extract year-month from the 'BALANCE_AS_OF_DATE' column
inventory_data['Month'] = inventory_data['BALANCE_AS_OF_DATE'].dt.to_period('M')

# Loop through each unique month and PLANT_NAME to save the data to new CSV files
for month, month_data in inventory_data.groupby('Month'):
    month_folder = os.path.join(output_directory, str(month))
    os.makedirs(month_folder, exist_ok=True)

    for plant, plant_data in month_data.groupby('PLANT_NAME'):
        plant_file_path = os.path.join(month_folder, f'{plant}_inventory.csv')
        plant_data.to_csv(plant_file_path, index=False)

# Verify the structure of the output directory
print(f"Data has been saved to: {output_directory}")


Data has been saved to: /content/Inventory


Sum Inventory my month

In [ ]:
import pandas as pd
import os

# ------------------------------------------------------------------
# 1)  CONFIGURE PATHS
# ------------------------------------------------------------------
inventory_csv = '/content/Inventory.csv'          # raw Inventory file
output_dir    = '/content/Inventory'      # where summary will live
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------------------
# 2)  LOAD & PREP
# ------------------------------------------------------------------
df = pd.read_csv(inventory_csv, parse_dates=['BALANCE_AS_OF_DATE'])

# Convert KG → metric tonnes (MT) if needed
df['UNRESRICTED_STOCK_MT'] = df['UNRESRICTED_STOCK'] / 1000.0

# Month key: YYYY-MM
df['Month'] = df['BALANCE_AS_OF_DATE'].dt.to_period('M').astype(str)

# ------------------------------------------------------------------
# 3)  SUM BY MONTH + WAREHOUSE
# ------------------------------------------------------------------
summary = (df.groupby(['Month', 'PLANT_NAME'])['UNRESRICTED_STOCK_MT']
             .sum()
             .reset_index()
             .rename(columns={
                 'PLANT_NAME': 'Warehouse',
                 'UNRESRICTED_STOCK_MT': 'Total_UNRESRICTED_STOCK_MT'
             })
             .sort_values(['Month', 'Warehouse'])
             .reset_index(drop=True))

# ------------------------------------------------------------------
# 4)  SAVE ONE MERGED FILE
# ------------------------------------------------------------------
output_file = os.path.join(output_dir, 'merged_inventory_summary.csv')
summary.to_csv(output_file, index=False)

print(f'✅  Summary created with {len(summary)} rows and saved to:\n{output_file}')


✅  Summary created with 26 rows and saved to:
/content/Inventory/merged_inventory_summary.csv


Inbound Date Classifying

In [ ]:
import pandas as pd
import os

# ------------------------------------------------------------------
# 1)  CONFIGURE PATHS
# ------------------------------------------------------------------
inbound_csv = '/content/Inbound.csv'         # raw file
output_dir  = '/content/Inbound'             # folder to drop the 2 files
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------------------
# 2)  LOAD DATA
# ------------------------------------------------------------------
df = pd.read_csv(inbound_csv, parse_dates=['INBOUND_DATE'])

# ------------------------------------------------------------------
# 3)  FIXED DATE RANGE  (01-Nov-2023  →  31-Jan-2025)
# ------------------------------------------------------------------
date_min = pd.Timestamp('2023-11-01')
date_max = pd.Timestamp('2025-01-31')
full_index = pd.date_range(date_min, date_max, freq='D')

# ------------------------------------------------------------------
# 4)  BUILD ONE DAILY-TOTAL FILE PER PLANT
# ------------------------------------------------------------------
for plant, g in df.groupby('PLANT_NAME'):
    daily_tot = (
        g.groupby(g['INBOUND_DATE'].dt.normalize())['NET_QUANTITY_MT']
          .sum()
          .rename('Total_NET_QUANTITY_MT')
          .reindex(full_index, fill_value=0)        # fill missing days
    )

    out_df = daily_tot.reset_index().rename(columns={'index': 'Date'})
    out_df.insert(1, 'Warehouse', plant)

    out_file = os.path.join(output_dir,
                            f'inbound_daily_totals_{plant}.csv')
    out_df.to_csv(out_file, index=False)
    print(f'✅  Saved {out_file}')

print('\n✔️  Files now cover 01-Nov-2023 → 31-Jan-2025 for each warehouse.')


✅  Saved /content/Inbound/inbound_daily_totals_CHINA-WAREHOUSE.csv
✅  Saved /content/Inbound/inbound_daily_totals_SINGAPORE-WAREHOUSE.csv

✔️  Files now cover 01-Nov-2023 → 31-Jan-2025 for each warehouse.


/tmp/ipython-input-4-1791026541.py:14: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(inbound_csv, parse_dates=['INBOUND_DATE'])


In [ ]:
import pandas as pd
import os
import re

# ------------------------------------------------------------------
# 1)  CONFIGURE PATHS
# ------------------------------------------------------------------
outbound_csv = '/content/Outbound.csv'     # raw outbound data
root_dir     = '/content/Outbound'         # master output folder

by_mode_dir      = os.path.join(root_dir, 'by_mode')
warehouse_all_dir = os.path.join(root_dir, 'warehouse_all')

os.makedirs(by_mode_dir, exist_ok=True)
os.makedirs(warehouse_all_dir, exist_ok=True)

# ------------------------------------------------------------------
# 2)  LOAD DATA
# ------------------------------------------------------------------
df = pd.read_csv(outbound_csv, parse_dates=['OUTBOUND_DATE'])
df.rename(columns=str.upper, inplace=True)   # uniform caps

# ------------------------------------------------------------------
# 3)  FIXED DATE WINDOW  (01-Nov-2023 → 31-Jan-2025)
# ------------------------------------------------------------------
date_min = pd.Timestamp('2023-11-01')
date_max = pd.Timestamp('2025-01-31')
full_index = pd.date_range(date_min, date_max, freq='D')

# ------------------------------------------------------------------
# 4)  A)  BY-MODE  (warehouse × transport mode)  →  4 files
# ------------------------------------------------------------------
for (plant, mode), g in df.groupby(['PLANT_NAME', 'MODE_OF_TRANSPORT']):
    daily_tot = (
        g.groupby(g['OUTBOUND_DATE'].dt.normalize())['NET_QUANTITY_MT']
          .sum()
          .rename('Total_NET_QUANTITY_MT')
          .reindex(full_index, fill_value=0)
    )

    out_df = daily_tot.reset_index().rename(columns={'index': 'Date'})
    out_df.insert(1, 'Warehouse', plant)
    out_df.insert(2, 'Transport_Mode', mode)

    safe_mode = re.sub(r'\W+', '_', mode).strip('_')
    file_path = os.path.join(by_mode_dir,
                             f'outbound_daily_totals_{plant}_{safe_mode}.csv')
    out_df.to_csv(file_path, index=False)
    print(f'✅  by_mode   → {file_path}')

# ------------------------------------------------------------------
# 4)  B)  WAREHOUSE-ALL (transport ignored)  →  2 files
# ------------------------------------------------------------------
for plant, g in df.groupby('PLANT_NAME'):
    daily_tot = (
        g.groupby(g['OUTBOUND_DATE'].dt.normalize())['NET_QUANTITY_MT']
          .sum()
          .rename('Total_NET_QUANTITY_MT')
          .reindex(full_index, fill_value=0)
    )

    out_df = daily_tot.reset_index().rename(columns={'index': 'Date'})
    out_df.insert(1, 'Warehouse', plant)

    file_path = os.path.join(warehouse_all_dir,
                             f'outbound_daily_totals_{plant}_ALL.csv')
    out_df.to_csv(file_path, index=False)
    print(f'✅  warehouse_all → {file_path}')

print('\n✔️  Done — transport-specific files in “by_mode/”, combined files in “warehouse_all/”.')


✅  by_mode   → /content/Outbound/by_mode/outbound_daily_totals_CHINA-WAREHOUSE_Truck.csv
✅  by_mode   → /content/Outbound/by_mode/outbound_daily_totals_SINGAPORE-WAREHOUSE_Marine.csv
✅  by_mode   → /content/Outbound/by_mode/outbound_daily_totals_SINGAPORE-WAREHOUSE_Truck.csv
✅  warehouse_all → /content/Outbound/warehouse_all/outbound_daily_totals_CHINA-WAREHOUSE_ALL.csv
✅  warehouse_all → /content/Outbound/warehouse_all/outbound_daily_totals_SINGAPORE-WAREHOUSE_ALL.csv

✔️  Done — transport-specific files in “by_mode/”, combined files in “warehouse_all/”.


In [ ]:
import pandas as pd
import os

# ------------------------------------------------------------------
# 1)  CONFIGURE PATHS
# ------------------------------------------------------------------
inbound_dir   = '/content/Inbound'                                          # daily inbound files
outbound_dir  = '/content/Outbound/warehouse_all'                           # daily outbound (ALL)
output_dir    = '/content/Inventory_Inbound_and_Outbound'                   # where to save the merged output
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------------------
# 2)  FUNCTION TO MERGE INBOUND AND OUTBOUND FOR A SPECIFIC WAREHOUSE
# ------------------------------------------------------------------
def merge_inbound_outbound(plant):
    # --- daily inbound (e.g., inbound_daily_totals_SINGAPORE-WAREHOUSE.csv) ------------------------
    in_file  = f'{inbound_dir}/inbound_daily_totals_{plant}-WAREHOUSE.csv'  # Corrected inbound file path
    inbound  = pd.read_csv(in_file, parse_dates=['Date'])

    # --- daily outbound (mode-agnostic) ----------------------------------------------
    out_file = f'{outbound_dir}/outbound_daily_totals_{plant}-WAREHOUSE_ALL.csv'  # Corrected outbound file path
    outbound = pd.read_csv(out_file, parse_dates=['Date'])

    # --- merge both datasets on 'Date' ----------------------------------------------
    merged = pd.merge(inbound, outbound, on='Date', how='outer', suffixes=('_Inbound', '_Outbound'))

    # --- fill missing data with 0 and calculate leftover
    merged['Inbound_MT'] = merged['Total_NET_QUANTITY_MT_Inbound'].fillna(0)
    merged['Outbound_MT'] = merged['Total_NET_QUANTITY_MT_Outbound'].fillna(0)

    # LeftOver = Inbound + (previous LeftOver) - Outbound
    merged['LeftOver_MT'] = merged['Inbound_MT'] - merged['Outbound_MT']

    # Add the 'Warehouse' column manually based on the `plant` name
    merged['Warehouse'] = plant

    # --- return merged DataFrame
    return merged[['Date', 'Warehouse', 'Inbound_MT', 'Outbound_MT', 'LeftOver_MT']]

# ------------------------------------------------------------------
# 3)  PROCESS BOTH WAREHOUSES
# ------------------------------------------------------------------
for warehouse in ['SINGAPORE', 'CHINA']:
    df_merged = merge_inbound_outbound(f'{warehouse}')  # Adjusted to match the correct naming pattern
    out_path = os.path.join(output_dir, f'Inventory_Inbound_and_Outbound_{warehouse}.csv')
    df_merged.to_csv(out_path, index=False)
    print(f'✅  Saved {out_path}')

print('\n✔️  Files saved successfully in /content/Inventory_Inbound_and_Outbound folder.')


✅  Saved /content/Inventory_Inbound_and_Outbound/Inventory_Inbound_and_Outbound_SINGAPORE.csv
✅  Saved /content/Inventory_Inbound_and_Outbound/Inventory_Inbound_and_Outbound_CHINA.csv

✔️  Files saved successfully in /content/Inventory_Inbound_and_Outbound folder.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter

# Load the data
url = 'https://docs.google.com/spreadsheets/d/1Y9UJwH5q6Xq6Xq6Xq6Xq6Xq6Xq6Xq6Xq6Xq6Xq6Xq6/export?format=xlsx'
df = pd.read_excel('cOst.xlsx')

# Convert date column to datetime
df['DATE'] = pd.to_datetime(df['DATE'])

# Create month column
df['Month'] = df['DATE'].dt.to_period('M')

# Calculate monthly sums
monthly_sgn = df.groupby('Month')['SGN_COST'].sum().reset_index()
monthly_cn = df.groupby('Month')['CN_COST'].sum().reset_index()
monthly_sgn['Month'] = monthly_sgn['Month'].astype(str)
monthly_cn['Month'] = monthly_cn['Month'].astype(str)

# Calculate averages
sgn_avg = df['SGN_COST'].mean()
cn_avg = df['CN_COST'].mean()

# Plot 1: Daily costs with averages and monthly bars
plt.figure(figsize=(20, 10))

# Plot monthly bars (background)
months = df['DATE'].dt.to_period('M').unique()
month_starts = [pd.Period(month, freq='M').start_time for month in months]
month_ends = [pd.Period(month, freq='M').end_time for month in months]

for start, end in zip(month_starts, month_ends):
    monthly_sgn_sum = df[(df['DATE'] >= start) & (df['DATE'] <= end)]['SGN_COST'].sum()
    monthly_cn_sum = df[(df['DATE'] >= start) & (df['DATE'] <= end)]['CN_COST'].sum()

    plt.bar(start, monthly_sgn_sum, width=25, color='lightblue', alpha=0.3, label='SGN Monthly' if start == month_starts[0] else "")
    plt.bar(start, monthly_cn_sum, width=25, color='lightgreen', alpha=0.3, label='CN Monthly' if start == month_starts[0] else "")

# Plot daily lines
plt.plot(df['DATE'], df['SGN_COST'], label='SGN Daily', color='blue')
plt.plot(df['DATE'], df['CN_COST'], label='CN Daily', color='green')

# Plot average lines
plt.axhline(y=sgn_avg, color='blue', linestyle='--', label=f'SGN Avg: {sgn_avg:,.2f}')
plt.axhline(y=cn_avg, color='green', linestyle='--', label=f'CN Avg: {cn_avg:,.2f}')

# Formatting
plt.title('Daily Costs with Monthly Totals and Averages')
plt.xlabel('Date')
plt.ylabel('Cost (THB)')
plt.legend()
plt.grid(True)

# Format y-axis to show full numbers
def millions(x, pos):
    return f'{x/1e6:.0f}M'
plt.gca().yaxis.set_major_formatter(FuncFormatter(millions))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot 2: Cumulative graph of both warehouses combined
plt.figure(figsize=(20, 10))

# Calculate combined cumulative
df['Combined_Cumulative'] = df['SGN_CUMULATIVE'] + df['CN_CUMULATIVE']

plt.plot(df['DATE'], df['Combined_Cumulative'], label='Combined Cumulative Cost', color='purple')

# Formatting
plt.title('Cumulative Cost of Both Warehouses')
plt.xlabel('Date')
plt.ylabel('Cumulative Cost (THB)')
plt.legend()
plt.grid(True)

# Format y-axis to show full numbers
def billions(x, pos):
    return f'{x/1e9:.1f}B'
plt.gca().yaxis.set_major_formatter(FuncFormatter(billions))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'cOst.xlsx'

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Load Inbound Data (already loaded earlier)
inbound_df['INBOUND_DATE'] = pd.to_datetime(inbound_df['INBOUND_DATE'])

# Filter data for relevant columns
inbound_data = inbound_df[['INBOUND_DATE', 'PLANT_NAME', 'MATERIAL_NAME', 'NET_QUANTITY_MT']]

# Feature Engineering: Extract month and year for seasonality
inbound_data['Year'] = inbound_data['INBOUND_DATE'].dt.year
inbound_data['Month'] = inbound_data['INBOUND_DATE'].dt.month
inbound_data['Day'] = inbound_data['INBOUND_DATE'].dt.day

# Creating lag features for previous month's inbound stock (Lag = 1)
inbound_data['Lag_1'] = inbound_data.groupby('MATERIAL_NAME')['NET_QUANTITY_MT'].shift(1)

# Drop rows with NaN values created by the lag feature
inbound_data = inbound_data.dropna()

# Filter data for training (Jan 2024 - Dec 2024)
train_data = inbound_data[inbound_data['Year'] == 2024]

# Define features and target variable
X = train_data[['Month', 'Lag_1']]
y = train_data['NET_QUANTITY_MT']

# Train-test split for validation (We will train on 2024 data and forecast Jan 2025)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost Model
model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                         max_depth=5, alpha=10, n_estimators=100)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error for Inbound Forecasting: {mae}')

# Forecast Inbound for January 2025
# Let's assume we are forecasting for one specific SKU (e.g., 'MAT-0045') and warehouse (e.g., 'SINGAPORE')
# We will forecast for January 2025 with known month and previous month's inbound (Lag_1)

forecast_data = {
    'Month': [1],  # January 2025
    'Lag_1': [inbound_data[inbound_data['MATERIAL_NAME'] == 'MAT-0045']['NET_QUANTITY_MT'].iloc[-1]]  # Use Dec 2024 inbound as Lag_1
}

forecast_df = pd.DataFrame(forecast_data)
forecasted_inbound = model.predict(forecast_df)

# Print the forecasted inbound quantity
print(f"Forecasted Inbound for Jan 2025 (MAT-0045): {forecasted_inbound[0]}")

# Plot the forecasted vs actual data (if actual data for Jan 2025 is available)
# This will help visualize how well the forecast performed

# Combine actual and forecasted data
plt.figure(figsize=(12, 6))
plt.plot(inbound_data['INBOUND_DATE'], inbound_data['NET_QUANTITY_MT'], label='Actual Inbound', color='blue')
plt.axvline(x=pd.to_datetime('2025-01-01'), color='red', linestyle='--', label='Forecast Start (Jan 2025)')

# Forecasting for January 2025
plt.scatter(pd.to_datetime('2025-01-01'), forecasted_inbound[0], color='green', label=f'Forecasted Jan 2025 (Inbound: {forecasted_inbound[0]:.2f} MT)')

plt.title('Inbound Forecast vs Actual (Jan 2025)')
plt.xlabel('Date')
plt.ylabel('Inbound Quantity (MT)')
plt.legend()
plt.grid(True)
plt.show()
